In [0]:
import os
foo = dbutils.secrets.get(scope = "my_scope", key = "POKEMON_API_KEY")
print(foo)

In [0]:
# %pip install pokemontcgsdk
import os
from pokemontcgsdk import Card, RestClient
from pyspark.sql import SparkSession
import pandas as pd

# Configure your API key from environment variable
api_key = dbutils.secrets.get(scope = "my_scope", key = "POKEMON_API_KEY")
if not api_key:
    raise ValueError("API key not found. Set the POKEMON_API_KEY environment variable.")
RestClient.configure(api_key)

# Fetch all cards using the SDK
cards = list(Card.all())

# Convert cards to a list of dicts
cards_list = [c.__dict__ for c in cards]

# Normalize complex fields
for card in cards_list:
    if 'abilities' in card:
        card['abilities'] = [ability.__dict__ for ability in card['abilities']]

# Create a pandas DataFrame
pdf = pd.json_normalize(cards_list)

# Convert to Spark DataFrame
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame(pdf)

# Examine schema and show data
df.printSchema()
display(df.limit(5))

# Write to Delta table
#df.write.format("delta") \
#    .mode("overwrite") \
#    .saveAsTable("pokemon_tcg_collection.bronze.tcg_all_cards")